In [3]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
# from scipy import stats
import matplotlib
import tikzplotlib
import os
import matplotlib.patches as mpatches
from scipy.stats import shapiro 
from scipy.stats import mannwhitneyu
import itertools

# Parametros de las representaciones graficas
sns.set(style="darkgrid")

matplotlib.rcParams["font.sans-serif"] = 'Times New Roman'
matplotlib.rcParams["font.serif"] = 'Times New Roman'
matplotlib.rcParams['axes.unicode_minus'] = False
matplotlib.rcParams['text.usetex'] = True


## Recogida de datos
pd.set_option('display.max_rows', 200)

results0 = pd.read_csv('results_0.csv').iloc[:, 10:]
results1 = pd.read_csv('results_1.csv').iloc[:, 10:]
results2 = pd.read_csv('results_2.csv').iloc[:, 10:]
results3 = pd.read_csv('results_3.csv').iloc[:, 10:]
results4 = pd.read_csv('results_4.csv').iloc[:, 10:]

parameters0 = pd.read_csv('parameters_0.csv').iloc[:, 2:]

results0 = pd.concat([parameters0, results0], axis=1)
results1 = pd.concat([parameters0, results1], axis=1)
results2 = pd.concat([parameters0, results2], axis=1)
results3 = pd.concat([parameters0, results3], axis=1)
results4 = pd.concat([parameters0, results4], axis=1)

results = pd.concat([results0, results1, results2, results3, results4]).reset_index()

results.rename(columns={'single':'approach'}, inplace=True)

results['perc_B'] = results['perc_B'].astype('category')
results['perc_B'] = results['perc_B'].cat.reorder_categories(['10.0%', '20.0%', '50.0%', '100%'])

results['k'] = results['k'].astype('category')
results['k'] = results['k'].cat.reorder_categories(['1', '10%', '25%'])

results['approach'] = results['approach'].astype('category')
results['approach'] = results['approach'].replace({True:'single', False:'multi'})
results['approach'] = results['approach'].cat.reorder_categories(['single', 'multi'])

results['init'] = results['init'].astype('category')
results['init'] = results['init'].replace({True:'with', False:'without'})
results['init'] = results['init'].cat.reorder_categories(['without', 'with'])

results['Gap'] *= 100
# results['ObjVal_h2'] = results[['ObjVal_h', 'ObjVal_h2']].min(axis=1)
# results.loc[results['init'] == True, 'Runtime'] = results.loc[results['init']== True, 'Runtime'].fillna(3600)
vector = ['Gap' , 'Runtime','NodeCount','ObjVal','Runtime_h','Runtime_h2','ObjVal_h','ObjVal_h2']

# results.loc[results['init'] == True, vector] = results.loc[results['init'] == True].groupby(['n_N', 'perc_B', 'k', 'approach', 'wL','lazy','A4']).transform(lambda x: x.fillna(x.mean())).reset_index().loc[:, vector]

results.loc[results['init'] == 'with', 'NodeCount'] = results.loc[results['init']== 'with', 'NodeCount'].ffill(limit=100)

results.loc[results['init'] == 'with', 'ObjVal_h'] = results.loc[results['init']== 'with', 'ObjVal_h'].ffill(limit=100)
results.loc[results['init'] == 'with', 'ObjVal_h2'] = results.loc[results['init']== 'with', ['ObjVal_h', 'ObjVal_h2']].min(axis=1)
results.loc[results['init'] == 'with', 'ObjVal'] = results.loc[results['init']== 'with', ['ObjVal', 'ObjVal_h', 'ObjVal_h2']].min(axis=1)
results.loc[results['init'] == 'with', 'Gap_math'] = 100*(results.loc[results['init']== 'with', 'ObjVal_h'] - results.loc[results['init']== 'with', 'ObjVal'])/results.loc[results['init']== 'with', 'ObjVal_h']
results.loc[results['init'] == 'with', 'Gap_build'] = 100*(results.loc[results['init']== 'with', 'ObjVal_h2'] - results.loc[results['init']== 'with', 'ObjVal_h'])/results.loc[results['init']== 'with', 'ObjVal_h2']
results.loc[results['init'] == 'with', 'Runtime_h'] = results.loc[results['init']== 'with', 'Runtime_h'].ffill(limit=100)
results.loc[results['init'] == 'with', 'Runtime_h2'] = results.loc[results['init']== 'with', 'Runtime_h2'].ffill(limit=100)

results.loc[results['init'] == 'with', 'Gap'] = results.loc[results['init']== 'with', 'Gap'].fillna(100)

results.to_csv('results_together.csv')

results_single = results.loc[results['approach'] == 'single']
results_multi = results.loc[results['approach'] == 'multi']

In [6]:
rows = list(map(lambda e: ('approach', e), list(results['approach'].unique()))) + list(map(lambda e: ('wL', e), list(results['wL'].unique()))) + list(map(lambda e: ('init', e), list(results['init'].unique()))) + list(map(lambda e: ('perc_B', e), list(results['perc_B'].unique()))) + list(map(lambda e: ('k', e), list(results['k'].unique())))
print(rows)

cols = list(map(lambda e: ('approach', e), list(itertools.combinations(results['approach'].unique(), 2))))
cols += list(map(lambda e: ('wL', e), list(itertools.combinations(results['wL'].unique(), 2))))
cols += list(map(lambda e: ('init', e), list(itertools.combinations(results['init'].unique(), 2))))
cols += list(map(lambda e: ('perc_B', e), list(itertools.combinations(results['perc_B'].unique(), 2))))
cols += list(map(lambda e: ('k', e), list(itertools.combinations(results['k'].unique(), 2))))

print(cols)

pvalues = {}

for row, value in rows:
    for col, test in cols:
        if row != col:
            value1, value2 = test
            pvalue = mannwhitneyu(x=np.array(results.loc[(results[row]==value) & (results[col] == value1), 'Runtime']), y=np.array(results.loc[(results[row]==value) & (results[col] == value2), 'Runtime']), alternative='less', nan_policy='omit').pvalue
            if pvalue < 0.05:
                pvalues[(row, value, col, str(value1) +'~' + str(value2))] = '< (' + str(np.round(pvalue, 3)) + ')'
            elif pvalue > 0.95:
                pvalues[(row, value, col, str(value1) +'~' + str(value2))] = '> (' + str(np.round(1 - pvalue, 3)) + ')'
            else:
                pvalues[(row, value, col, str(value1) +'~' + str(value2))] = 'no difference'



df = pd.Series(pvalues).reset_index()

df = df.pivot(index=['level_0', 'level_1'], columns=['level_2', 'level_3'])

df.to_excel('table_pvalue_runtime.xlsx')

[('approach', 'single'), ('approach', 'multi'), ('wL', 0), ('wL', 50), ('init', 'without'), ('init', 'with'), ('perc_B', '10.0%'), ('perc_B', '20.0%'), ('perc_B', '50.0%'), ('perc_B', '100%'), ('k', '1'), ('k', '10%'), ('k', '25%')]
[('approach', ('single', 'multi')), ('wL', (0, 50)), ('init', ('without', 'with')), ('perc_B', ('10.0%', '20.0%')), ('perc_B', ('10.0%', '50.0%')), ('perc_B', ('10.0%', '100%')), ('perc_B', ('20.0%', '50.0%')), ('perc_B', ('20.0%', '100%')), ('perc_B', ('50.0%', '100%')), ('k', ('1', '10%')), ('k', ('1', '25%')), ('k', ('10%', '25%'))]


In [2]:
results_init = results.loc[results['init'] == 'with']
print(results_init)

rows = list(map(lambda e: ('approach', e), list(results_init['approach'].unique()))) + list(map(lambda e: ('wL', e), list(results_init['wL'].unique()))) + list(map(lambda e: ('perc_B', e), list(results_init['perc_B'].unique()))) + list(map(lambda e: ('k', e), list(results_init['k'].unique())))
print(rows)

cols = list(map(lambda e: ('approach', e), list(itertools.combinations(results_init['approach'].unique(), 2))))
cols += list(map(lambda e: ('wL', e), list(itertools.combinations(results_init['wL'].unique(), 2))))
cols += list(map(lambda e: ('perc_B', e), list(itertools.combinations(results_init['perc_B'].unique(), 2))))
cols += list(map(lambda e: ('k', e), list(itertools.combinations(results_init['k'].unique(), 2))))

print(cols)

pvalues = {}

for row, value in rows:
    for col, test in cols:
        if row != col:
            value1, value2 = test
            pvalue = mannwhitneyu(x=np.array(results_init.loc[(results_init[row]==value) & (results_init[col] == value1), 'Gap_build']), y=np.array(results_init.loc[(results_init[row]==value) & (results_init[col] == value2), 'Gap_build']), alternative='less', nan_policy='omit').pvalue
            if pvalue < 0.05:
                pvalues[(row, value, col, str(value1) +'~' + str(value2))] = '< (' + str(np.round(pvalue, 3)) + ')'
            elif pvalue > 0.95:
                pvalues[(row, value, col, str(value1) +'~' + str(value2))] = '> (' + str(np.round(1 - pvalue, 3)) + ')'
            else:
                pvalues[(row, value, col, str(value1) +'~' + str(value2))] = 'no difference'



df = pd.Series(pvalues).reset_index()

df = df.pivot(index=['level_0', 'level_1'], columns=['level_2', 'level_3'])

df.to_excel('table_pvalue_gap_build.xlsx')

      index  n_N perc_B    k approach  wL   lazy     A4  init         Gap  \
1         1   10  10.0%    1   single   0  False  False  with    0.006841   
3         3   10  20.0%    1   single   0  False  False  with    0.001106   
5         5   10  50.0%    1   single   0  False  False  with    0.000771   
7         7   10   100%    1   single   0  False   True  with    0.000000   
9         9   10  10.0%    1   single  50  False  False  with    0.008498   
...     ...  ...    ...  ...      ...  ..    ...    ...   ...         ...   
2391    471   70   100%  25%    multi   0  False   True  with  100.000000   
2393    473   70  10.0%  25%    multi  50  False  False  with  100.000000   
2395    475   70  20.0%  25%    multi  50  False  False  with  100.000000   
2397    477   70  50.0%  25%    multi  50  False  False  with  100.000000   
2399    479   70   100%  25%    multi  50  False   True  with  100.000000   

         Runtime  NodeCount       ObjVal   Runtime_h  Runtime_h2     ObjVal